# Quantum RNN

Partes do código do repo: [AgentANAKIN Q-RNN](https://github.com/AgentANAKIN/Quantum-Neural-Network "Quantum Neural Network"), que propõe uma rede neural simples quântica, possível de ser usada como base para maiores implementações.

### Configurações 

Para configurar a rede base para uso no **IBM Quantum Experience**:

- Usar o API token como variável de ambiente;
- Salvar o token para não precisar reiniciar;
- De preferência usar uma virtualenv.

In [1]:
import os

from qiskit import Aer, ClassicalRegister, execute, QuantumCircuit, QuantumRegister, IBMQ
from qiskit.tools.monitor import job_monitor
from numpy import exp, array, random, dot

API_TOKEN = os.getenv('API_TOKEN')
IBMQ.save_account(API_TOKEN)

Credentials already present. Set overwrite=True to overwrite.


### Métodos para medição

Com o código a baixo é possível "setar" os backends, ou para real, ou para simulado. O simulado é o mesmo utilzado nos códigos para coloração de grafos que eu implementei no outro notebook. Mas para esse aqui, eu coloquei para rodar em backends reais nas localidades disponíveis para minha conta no IBM Q Experience.

- Primeiro verifica se o backend escolhido está disponível;
- Depois são configurados os medidores;
- Por fim, é feita a conversão das medições;

In [2]:
def quniform(min, max):
    range = max - min
    qaddend = range * qmeasure('sim')
    qsum = qaddend + min
    return qsum

def qquniform(min, max):
    range = max - min
    qaddend = range * qmeasure('real')
    qsum = qaddend + min
    return qsum

def qmeasure(hardware):
    if (hardware == 'real'):
        qubits = 14
        #from qiskit.providers.ibmq import least_busy
        #backend = least_busy(IBMQ.backends())
        provider = IBMQ.load_account()
        backend = provider.get_backend('ibmq_16_melbourne')
    else:
        qubits = 32
        backend = Aer.get_backend('qasm_simulator')
        
    q = QuantumRegister(qubits) 
    c = ClassicalRegister(qubits) 
    qc = QuantumCircuit(q, c) # initialize the circuit

    i = 0
    while i < qubits:
        qc.h(q[i]) # put all qubits into superposition states
        i = i + 1
   
    qc.measure(q, c) # collapse the superpositions and get random zeroes and ones
    job = execute(qc, backend=backend, shots=1)
    job_monitor(job)
    result = job.result()
    mraw = result.get_counts(qc)
    m = str(mraw)
    subtotal = 0
    for i in range(qubits):
        subtotal = subtotal + (int(m[i+2]) * 2**(i)) # convert each binary digit to its decimal value, but read left-to-right for simplicity
    multiplier = subtotal / (2**qubits) # convert the measurement to a value between 0 and 1
    
    return multiplier

### Rede Neural

A rede neural calculada é **simples**, e utiliza fórmula básica para calculo dos pesos de acordo com a quântidade de entradas. É uma função com características lineares:
\
![E_formula](./img/formula.png)
\
O retorno que virá do processamento quântico serão os pesos calculados para ela após o treino.
\
\
**Info extra:** De acordo com essa imagem aqui e com as referências:
\
![RNN_Deep_RNN](./img/net.png)
\
Uma rede profunda é aquela que possue múltiplas camadas de características não-lineares, segundo Alexey Grigoryevich Ivakhnenko (desenvolveu o Método do Grupo de Manipulação de Dados) e Valentin Grigor’evich Lapa (autor de Cybernetics and Forecasting Techniques) em 1965.

In [3]:
class neural_network:
    def __init__(self):
        self.weights = []
        self.weights.append([qquniform(-1, 1)])
        self.weights.append([qquniform(-1, 1)])
        print("self.weights ",self.weights)

    def train(self, inputs, outputs, num):
        for iteration in range(num):
            output = self.think(inputs)
            error = outputs - output
            adjustment = 0.01*dot(inputs.T, error)
            self.weights += adjustment

    def think(self, inputs):
        return (dot(inputs, self.weights))


### Run 
- Um conjunto de entradas para um dado array de saída aleatório

In [4]:
neural_network = neural_network()

# training data
inputs = array([[2, 3], [1, 1], [5, 2], [12, 3]])
outputs = array([[10, 4, 14, 30]]).T

neural_network.train(inputs, outputs, 10000)

print(neural_network.think(array([15, 2])))

/Users/flaviajanine/.virtualenvs/qiskit_VQE/lib/python3.6/site-packages/qiskit/providers/models/backendconfiguration.py:337: UserWarning: `dt` and `dtm` now have units of seconds(s) rather than nanoseconds(ns).
  warnings.warn('`dt` and `dtm` now have units of seconds(s) rather '


Job Status: job has successfully run


Credentials are already in use. The existing account in the session will be replaced.


Job Status: job has successfully run
self.weights  [[0.25537109375], [-0.5458984375]]
[34.]


### Resultados

- O resultado acima foi para o circuito com 1 qbit:
\
![singleQ](./img/single.png)

- Job Status: job has successfully run

```python
Self.weights: [[0.0], [-1.0]]
Output: [14.]
```
- Gerou os pesos listados acima, em formato de array de tuplas, como resultado do Job "print"

- Para o circuito com 5 qbits:
\
![fiveQ](./img/five.png)

- O array com pesos foi retornado:
\
![resultfiveQ](./img/resultfive.jpeg)
